## Business Recommendation System

Adaptated from: 
- https://www.kaggle.com/code/sagarbapodara/coursera-course-recommendation-system-webapp

In [64]:
import os
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity as cs

import nltk
from nltk.stem.porter import PorterStemmer

In [65]:
df = pd.read_csv("user_input.csv")

In [66]:
print(type(df.bidang_keahlian[0]))

df.bidang_keahlian = df.bidang_keahlian.apply(lambda x: x.replace("[","").replace("]","").replace("'",""))
df.hobi = df.hobi.apply(lambda x: x.replace("[","").replace("]","").replace("'",""))

df

<class 'str'>


,user_id,level,bidang_keahlian,hobi,modal_usaha
0,0,udah_punya_usaha,"kuliner, healtcare","baca, travelling",above_100
1,1,udah_punya_usaha,"kuliner, healtcare","nonton, olahraga",under_50
2,2,udah_punya_usaha,"healtcare, kuliner","travelling, olahraga, baca",between_50_and_100
3,3,udah_punya_usaha,kuliner,"membaca, travelling, nonton, baca",under_50
4,4,udah_punya_usaha,healtcare,travelling,above_100
5,5,udah_punya_usaha,"kuliner, healtcare, homecare","baca, nonton",above_100
6,6,udah_punya_usaha,healtcare,"nonton, baca",above_100
7,7,udah_punya_usaha,"healtcare, homecare, kuliner","makan, baca, membaca, nonton, olahraga",between_50_and_100
8,8,udah_punya_usaha,"homecare, healtcare","membaca, makan",under_50
9,9,udah_punya_usaha,homecare,"makan, olahraga, travelling, baca, nonton",under_50


In [67]:
df.info()
df.isnull().sum()
df.level
df.bidang_keahlian
df.hobi
df.modal_usaha

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   user_id          50 non-null     int64 
 1   level            50 non-null     object
 2   bidang_keahlian  50 non-null     object
 3   hobi             50 non-null     object
 4   modal_usaha      50 non-null     object
dtypes: int64(1), object(4)
memory usage: 2.1+ KB


0              above_100
1               under_50
2     between_50_and_100
3               under_50
4              above_100
5              above_100
6              above_100
7     between_50_and_100
8               under_50
9               under_50
10    between_50_and_100
11    between_50_and_100
12              under_50
13    between_50_and_100
14             above_100
15              under_50
16             above_100
17             above_100
18             above_100
19              under_50
20              under_50
21              under_50
22    between_50_and_100
23    between_50_and_100
24    between_50_and_100
25    between_50_and_100
26    between_50_and_100
27              under_50
28             above_100
29             above_100
30    between_50_and_100
31             above_100
32              under_50
33             above_100
34    between_50_and_100
35    between_50_and_100
36    between_50_and_100
37             above_100
38              under_50
39              under_50


## Preprocessing

In [68]:
df.user_id= df.user_id.astype(str)

## Tage Column

In [69]:
df["features"] = df["bidang_keahlian"] + "," + df["hobi"]  + "," + df["modal_usaha"]
df

,user_id,level,bidang_keahlian,hobi,modal_usaha,features
0,0,udah_punya_usaha,"kuliner, healtcare","baca, travelling",above_100,"kuliner, healtcare,baca, travelling,above_100"
1,1,udah_punya_usaha,"kuliner, healtcare","nonton, olahraga",under_50,"kuliner, healtcare,nonton, olahraga,under_50"
2,2,udah_punya_usaha,"healtcare, kuliner","travelling, olahraga, baca",between_50_and_100,"healtcare, kuliner,travelling, olahraga, baca,..."
3,3,udah_punya_usaha,kuliner,"membaca, travelling, nonton, baca",under_50,"kuliner,membaca, travelling, nonton, baca,unde..."
4,4,udah_punya_usaha,healtcare,travelling,above_100,"healtcare,travelling,above_100"
5,5,udah_punya_usaha,"kuliner, healtcare, homecare","baca, nonton",above_100,"kuliner, healtcare, homecare,baca, nonton,abov..."
6,6,udah_punya_usaha,healtcare,"nonton, baca",above_100,"healtcare,nonton, baca,above_100"
7,7,udah_punya_usaha,"healtcare, homecare, kuliner","makan, baca, membaca, nonton, olahraga",between_50_and_100,"healtcare, homecare, kuliner,makan, baca, memb..."
8,8,udah_punya_usaha,"homecare, healtcare","membaca, makan",under_50,"homecare, healtcare,membaca, makan,under_50"
9,9,udah_punya_usaha,homecare,"makan, olahraga, travelling, baca, nonton",under_50,"homecare,makan, olahraga, travelling, baca, no..."


In [70]:
new_df = df[["user_id","features"]]
new_df.features = new_df.features.apply(lambda x: x.replace(","," "))
new_df.features = new_df.features.apply(lambda x: x.lower())

new_df

C:\Users\Muhammad Nur Ilmi\AppData\Local\Temp\ipykernel_9264\3855829668.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.features = new_df.features.apply(lambda x: x.replace(","," "))
C:\Users\Muhammad Nur Ilmi\AppData\Local\Temp\ipykernel_9264\3855829668.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.features = new_df.features.apply(lambda x: x.lower())


,user_id,features
0,0,kuliner healtcare baca travelling above_100
1,1,kuliner healtcare nonton olahraga under_50
2,2,healtcare kuliner travelling olahraga baca ...
3,3,kuliner membaca travelling nonton baca unde...
4,4,healtcare travelling above_100
5,5,kuliner healtcare homecare baca nonton abov...
6,6,healtcare nonton baca above_100
7,7,healtcare homecare kuliner makan baca memb...
8,8,homecare healtcare membaca makan under_50
9,9,homecare makan olahraga travelling baca no...


In [71]:
new_df.shape

(50, 2)

In [72]:
cv = CountVectorizer(max_features = 5000)
vectors = cv.fit_transform(new_df.features).toarray()

In [73]:
porterStemmer = PorterStemmer()
def stem(txt):
    y = []
    for t in txt.split():
        y.append(porterStemmer.stem(t))
    return " ".join(y)

In [74]:
new_df.features = new_df.features.apply(stem)

C:\Users\Muhammad Nur Ilmi\AppData\Local\Temp\ipykernel_9264\1174626138.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.features = new_df.features.apply(stem)


In [75]:
similarityDistance = cs(vectors)
print(similarityDistance)

[[1.         0.4        0.73029674 ... 0.59628479 0.25819889 0.36514837]
 [0.4        1.         0.54772256 ... 0.4472136  0.51639778 0.73029674]
 [0.73029674 0.54772256 1.         ... 0.68041382 0.70710678 0.5       ]
 ...
 [0.59628479 0.4472136  0.68041382 ... 1.         0.38490018 0.54433105]
 [0.25819889 0.51639778 0.70710678 ... 0.38490018 1.         0.47140452]
 [0.36514837 0.73029674 0.5        ... 0.54433105 0.47140452 1.        ]]


In [82]:
def getRecommendation(user_id):
    index = new_df[new_df["user_id"] == user_id].index[0]
    distance = similarityDistance[index]
    userRank = sorted(list(enumerate(distance)), reverse = True, key = lambda x:x[1])[1:5]

    for i in userRank:
        print(i)
        # print(new_df.iloc[i])

In [83]:
getRecommendation("0")

(29, 0.7999999999999999)
(41, 0.7999999999999999)
(16, 0.7905694150420948)
(4, 0.7745966692414835)
